Useful links:


- [AZ Py SDK Hyperparameter Tuning](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#sample-notebook)


- [AZ Py SDK Hyperparameter Notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb)


In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = "udacity-project"

exp = Experiment(workspace=ws, name= experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-161250
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-161250


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###



cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)


cpu_cluster.wait_for_completion(show_output=True)

    
    


InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os



# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.01,0.5),
    "--max_iter": choice(100,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,
                      slack_factor=0.3,
                      delay_evaluation=3)


# * create a training folder
project_folder = "./training"
if "training" not in os.listdir():
    os.mkdir(project_folder)



In [4]:
# Create a SKLearn estimator for use with train.py


est = SKLearn(source_directory='./',
              compute_target=cpu_cluster,
              entry_script='train.py',
              script_params={'--C':1,'--max_iter':100})



# Policy not working
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     policy=policy,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()


hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d1c501a3-013a-45ca-ba02-d8db59def310
Web View: https://ml.azure.com/runs/HD_d1c501a3-013a-45ca-ba02-d8db59def310?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-161250/workspaces/quick-starts-ws-161250&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-17T16:17:34.940682][API][INFO]Experiment created<END>\n""<START>[2021-10-17T16:17:35.384077][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-10-17T16:17:35.551494][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d1c501a3-013a-45ca-ba02-d8db59def310
Web View: https://ml.azure.com/runs/HD_d1c501a3-013a-45ca-ba02-d8db59def310?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-161250/workspaces/quick-starts-ws-161250&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



In [6]:
# Get your best run and save the model from that run.



best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.id)

best_run_metric = best_run.get_metrics()
print(best_run_metric['Accuracy'])



model = best_run.register_model(model_name='LogReg_HyperTuned', model_path="logs/azureml") 

['--C', '0.26188126013001023', '--max_iter', '100']
HD_d1c501a3-013a-45ca-ba02-d8db59def310_1
0.9111785533636824


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_aml = TabularDatasetFactory.from_delimited_files(path = web_path)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x,y = clean_data(data_aml)

import pandas as pd

dataset = pd.concat([x,y],axis=1)

In [9]:
from azureml.core import Dataset

dataset.to_csv('data.csv',index = False)

datastore = ws.get_default_datastore()

datastore.upload_files(files = ['./data.csv'])

data = Dataset.Tabular.from_delimited_files([(datastore,'data.csv')])

Uploading an estimated of 1 files
Uploading ./data.csv
Uploaded ./data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [10]:
from azureml.train.automl import AutoMLConfig
import logging

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4, # This is a limit for testing purpose, please increase it as per cluster size
    "experiment_timeout_minutes": 30, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             max_cores_per_iteration=-1,
                             compute_target = cpu_cluster,
                             training_data = data,
                             label_column_name = "y",
                             **automl_settings
                            )

In [11]:
# Submit your automl run

exp_aml = Experiment(ws, "aml")

aml_run = exp_aml.submit(automl_config, show_output = True)

from azureml.widgets import RunDetails
RunDetails(aml_run).show()

Submitting remote run.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster


Experiment,Id,Type,Status,Details Page,Docs Page
aml,AutoML_aa6f2304-9dad-40e3-9aeb-1588f08f8ba9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, aml_model = aml_run.get_output()

print(best_run)
print(aml_model)

best_run.register_model(model_name='AML', model_path="logs/azureml")

Run(Experiment: aml,
Id: AutoML_aa6f2304-9dad-40e3-9aeb-1588f08f8ba9_36,
Type: azureml.scriptrun,
Status: Completed)
None


Model(workspace=Workspace.create(name='quick-starts-ws-161250', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-161250'), name=AML, id=AML:1, version=1, tags={}, properties={})

In [16]:
print(best_run)

Run(Experiment: aml,
Id: AutoML_aa6f2304-9dad-40e3-9aeb-1588f08f8ba9_36,
Type: azureml.scriptrun,
Status: Completed)
Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [13]:
cpu_cluster.delete()